# `Memgraph` & `NetworkX` Report
Comparison of the `get_paths()` feature in `memgraph` vs `networkx`

In [6]:
import time

---  
## `Memgraph`  

In [7]:
import mgclient # connectto memgraph client


In [14]:
# connect to the memgraph server
conn = mgclient.connect(host="localhost", port=7687)   # connect to memgraph
conn.autocommit = True  # autocommit mode is required for DDL queries
cursor = conn.cursor() # cursor is used to execute queries

In [21]:
# # # Delete all nodes and relationships
cursor.execute("MATCH (n) DETACH DELETE n")

In [15]:
def get_time(start_time):
    # End timing
    end_time = time.time()

    # Calculate duration in seconds
    duration_seconds = end_time - start_time

    # Check if duration is 60 seconds or more
    if duration_seconds >= 60:
        # Convert to minutes and round to two decimal places
        duration_minutes = round(duration_seconds / 60, 2)
        print(f"Execution time: {duration_minutes} minutes")
    else:
        # Round to two decimal places for seconds
        rounded_seconds = round(duration_seconds, 2)
        print(f"Execution time: {rounded_seconds} seconds")

def print_memgraph_paths_results(results):
    for result in results:
        path = result[0]  # Get the path from the result

        # Print the full path
        full_path = " -> ".join(node.properties['name'] for node in path.nodes)
        print("Full Path:", full_path)

        # Iterate over the nodes to print predicates for each segment
        for i in range(len(path.nodes) - 1):
            start_node_name = path.nodes[i].properties['name']
            end_node_name = path.nodes[i + 1].properties['name']

            # Assuming the predicates are stored in the relationships in the same sequence
            predicates = path.relationships[i].properties.get('predicates', [])

            print(f"{start_node_name} - {end_node_name}: {predicates}")

        print("--------------------------------------------------\n")


View Index Info

In [22]:
# Index on the name property of the Entity node
q="""
CREATE INDEX ON :Entity(name);
"""

cursor.execute(q) # execute query

In [17]:
cursor.execute("""SHOW INDEX INFO;""")
cursor.fetchall()

[('label+property', 'Entity', 'name', 113)]

In [19]:
cursor.execute("""ANALYZE GRAPH;""")
cursor.fetchall()

[('Entity', 'name', 113, 113, 1.0, 0.0, 111.80530973451327)]

---
The Algorithm:
```
path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..5]->(end:Entity {name: 'Publication'})
RETURN p;
"""
start_time= time.time()
cursor.execute(path_query)
mg_5h_results = cursor.fetchall()
print(f"Number of results(4-hop): {len(mg_5h_results)}")
get_time(start_time)
```

Get a set of 100 

In [5]:
# Start timing
start_time = time.time()

path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..3]->(end:Entity {name: 'Publication'})
RETURN p
LIMIT 100;
"""
cursor.execute(path_query)
results = cursor.fetchall()


print(f"Number of results: {len(results)}")

get_time(start_time)


Number of results: 100
Execution time: 0.64 seconds


Print full set of three

In [6]:
path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..3]->(end:Entity {name: 'Publication'})
RETURN p;
"""

start_time= time.time()
cursor.execute(path_query)
mg_3h_results = cursor.fetchall()
print(f"Number of results(3-hop): {len(mg_3h_results)}")
get_time(start_time)



Number of results(3-hop): 531
Execution time: 1.15 seconds


4-hop

In [7]:
path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..4]->(end:Entity {name: 'Publication'})
RETURN p;
"""
start_time= time.time()
cursor.execute(path_query)
mg_4h_results = cursor.fetchall()
print(f"Number of results(4-hop): {len(mg_4h_results)}")
get_time(start_time)


Number of results(4-hop): 37205
Execution time: 2.8 minutes


5-hop

In [ ]:
path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..5]->(end:Entity {name: 'Publication'})
RETURN p
LIMIT 100;
"""
start_time= time.time()
cursor.execute(path_query)
mg_5h_results = cursor.fetchall()
print(f"Number of results(5-hop): {len(mg_5h_results)}")
get_time(start_time)


### `Memgraph` with `algo` 

In [10]:
query="""
MATCH (start:Entity {name: 'InformationResource'}) MATCH (end:Entity {name: 'Publication'})
CALL algo.all_simple_paths(start, end, [], 3) 
YIELD path AS result RETURN result;
"""
start_time= time.time()
cursor.execute(query)
algo_3h_results = cursor.fetchall()
print(f"Number of results(3-hop): {len(algo_3h_results)}")
get_time(start_time)


Number of results(3-hop): 2547
Execution time: 5.2 seconds


In [11]:
algo_3h_results[0]

(<mgclient.Path(nodes=[<mgclient.Node(id=449, labels={'Entity'}, properties={'name': 'InformationResource'}) at 0x119809dd0>, <mgclient.Node(id=391, labels={'Entity'}, properties={'name': 'Behavior'}) at 0x1198b8f60>, <mgclient.Node(id=362, labels={'Entity'}, properties={'name': 'MolecularEntity'}) at 0x111100ff0>, <mgclient.Node(id=344, labels={'Entity'}, properties={'name': 'Publication'}) at 0x111103900>], relationships=[<mgclient.Relationship(start_id=391, end_id=449, type='RELATED_TO', properties={'api': [[{'bte': {'query_operation': {'input_separator': ',', 'method': 'post', 'params': None, 'path': '/query', 'path_params': None, 'request_body': None, 'server': 'https://kg2.transltr.io/api/rtxkg2/v1.3', 'support_batch': True}}, 'name': 'RTX KG2 - TRAPI 1.3.0', 'smartapi': {'id': 'ccd4a8bb83de81401e9a27f1d8e7f948', 'metadata': 'https://raw.githubusercontent.com/RTXteam/RTX/production/code/UI/OpenAPI/python-flask-server/KG2/openapi_server/openapi/openapi.yaml', 'ui': 'https://smart-

---

## NetworkX

In [4]:
from utils.metakg.path_finder import MetaKGPathFinder
import networkx.classes.digraph as nx_digraph
import networkx as nx


In [7]:
subject='InformationResource'
object='Publication'

start_time=time.time()
query_data = {
    'q': '*'
}
pf = MetaKGPathFinder(query_data)


In [13]:
type(pf)

utils.metakg.path_finder.MetaKGPathFinder

In [10]:

G = pf.G
cutoff=3
nx_3h_results = pf.get_paths(subject=subject, object=object, cutoff=cutoff)
print(f"Number of results(3-hop): {len(nx_3h_results)}")
get_time(start_time)

Number of results(3-hop): 448
Execution time: 23.53 seconds


In [12]:
start_time=time.time()
G = pf.G
cutoff=4
nx_4h_results = pf.get_paths(subject=subject, object=object, cutoff=cutoff)
print(f"Number of results(4-hop): {len(nx_4h_results)}")
get_time(start_time)

Number of results(4-hop): 30776
Execution time: 1.17 minutes


In [14]:
start_time=time.time()
G = pf.G
cutoff=5
nx_5h_results = pf.get_paths(subject=subject, object=object, cutoff=cutoff)
print(f"Number of results(5-hop): {len(nx_5h_results)}")
get_time(start_time)

---